In [2]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import time
import json


In [3]:
load_dotenv()

True

In [4]:
test_address = 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU'
test_wallet_timeseries = pd.read_csv('../data/processed/test_acc_timeseries.csv')

In [5]:
test_wallet_timeseries['timestamp'] = pd.to_datetime(test_wallet_timeseries['timestamp'])
test_wallet_timeseries = test_wallet_timeseries.sort_values(by='timestamp',ascending=False)
test_wallet_timeseries['sender'].head()
unique_receivers = test_wallet_timeseries['receiver'].dropna().unique()
unique_senders = test_wallet_timeseries['sender'].dropna().unique()
unique_addresses = list(set(unique_receivers.tolist() + unique_senders.tolist()))
len(unique_addresses)

241

# Flipside Labels

https://flipsidecrypto.xyz/studio/queries/d238f65d-0fdf-4f56-b7df-6ec6d95a0559

In [6]:
flipside_files = ['solana_cex_labels','solana_chadmin','solana_dapp_labels','solana_defi_labels']
flipside_labels_df = pd.DataFrame()
for file in flipside_files:
    dir = f'{file}.csv'
    df = pd.read_csv(f"../data/raw/{file}.csv", on_bad_lines='skip').dropna()
    flipside_labels_df = pd.concat([flipside_labels_df,df])


In [7]:
flipside_labels_df = flipside_labels_df.drop_duplicates(subset='ADDRESS')
flipside_labels_df

,ADDRESS,LABEL_TYPE,LABEL_SUBTYPE,ADDRESS_NAME,PROJECT_NAME
0,F7nqmScTgLEGqB13dg2VQeURpk7htKvkdcxK5fXZGoqk,cex,deposit_wallet,kucoin deposit_wallet,kucoin
1,DN8hF8iv1jmktXdYAfs2mAkoLycpYimU7xE5LWA8umrr,cex,deposit_wallet,coinbase deposit_wallet,coinbase
2,Dw9wBygiwVWF9DHQFoEYPb5AqJmBBEcdiJnkhDYajFTd,cex,deposit_wallet,ftx deposit_wallet,ftx
3,DZJEkYa9zoMYrMkwhvh3Qen5ER8Ga6voRDJDCZBR7Ykz,cex,deposit_wallet,binance deposit_wallet,binance
4,58z1n26qxUdHukASYvsk7VoHMNH4LDxWMvouXVVi5rXj,cex,deposit_wallet,backpack exchange deposit_wallet,backpack exchange
...,...,...,...,...,...
1474,H8h7ZyS5qJR2cwLxvZQdPaNzLik17cRxB5pDvjdXeuBg,defi,router,kamino: sol/wif strategy address,kamino
1475,12kG3nQHrp9hozgBS2PYAKdo5SMtJRELfKYjjwvwrbhX,defi,pool,frakt: taiyo pilots pool,frakt
1476,8KvNpeyKyZYWfHsBtds4fRLHXRaMU9aop43eagXffdU,defi,general_contract,upfi,upfi
1477,2hdg3iYYz4ZkyhUVQU4uzQybiCEfkQ6kYvMuGmtXUYg2,defi,pool,frakt: communi3: mad scientists pool,frakt


In [8]:
flipside_labels_dict = dict(zip(flipside_labels_df['ADDRESS'], flipside_labels_df['ADDRESS_NAME']))
flipside_labels_dict

{'F7nqmScTgLEGqB13dg2VQeURpk7htKvkdcxK5fXZGoqk': 'kucoin deposit_wallet',
 'DN8hF8iv1jmktXdYAfs2mAkoLycpYimU7xE5LWA8umrr': 'coinbase deposit_wallet',
 'Dw9wBygiwVWF9DHQFoEYPb5AqJmBBEcdiJnkhDYajFTd': 'ftx deposit_wallet',
 'DZJEkYa9zoMYrMkwhvh3Qen5ER8Ga6voRDJDCZBR7Ykz': 'binance deposit_wallet',
 '58z1n26qxUdHukASYvsk7VoHMNH4LDxWMvouXVVi5rXj': 'backpack exchange deposit_wallet',
 '94Pzars8ccjgiGgFJ3y6SGXBx7FC3JjBSDL34sArVJZk': 'kraken deposit_wallet',
 'FSUsvp5wk9BwHHQMLcWVHgb1hKFuXSb5RG6suuKi1294': 'backpack exchange deposit_wallet',
 'fznuAN32RGr2CpYUhSqHBb9v3LaiBHJFDSksZGV4NzL': 'coinbase deposit_wallet',
 'Au7gWXZSwQdULT6NN82JaE8FUeNKrvaZAZakWAcVbUoX': 'kucoin deposit_wallet',
 'BBiAMmmo9QtsW6gfzPjxEngNxkbf9RDNNQa92egk1Q3J': 'backpack exchange deposit_wallet',
 'FkDzh9GMam7bLPDqsh3kdcwvvFGCdfKqsSqumTNxfXLH': 'bybit deposit_wallet',
 '6TbjfKvdhihs192ZTHXBpRgoHyzgNdy18K13aEJUpgs3': 'gate.io deposit_wallet',
 '6ddL3YL9GEB1AFa5cyMs7ga2WC33CATpVrTcysDKzPNb': 'mexc deposit_wallet',
 '29ev

# Vybe

In [9]:
VYBE_API_KEY = os.getenv('VYBE_API_KEY')

In [10]:
# Ideally we can cache this weekly; likely doesnt change too frequently

def get_vybe_identified_accounts(use_cache=True):

    url = "https://api.vybenetwork.xyz/account/known-accounts"

    headers = {"accept": "application/json","X-API-KEY":VYBE_API_KEY}

    response = requests.get(url, headers=headers)

    data = response.json()

    identified_addresses = {}
    for account in data['accounts']:
        address = account.get('ownerAddress')
        address = address.lower()
        name = account.get('name')
        identified_addresses[address] = name

    return identified_addresses

In [11]:
# Ideally we can cache this weekly; likely doesnt change too frequently

def get_vybe_identified_programs(use_cache=True):

    url = "https://api.vybenetwork.xyz/program/known-program-accounts"

    headers = {"accept": "application/json","X-API-KEY":VYBE_API_KEY}

    response = requests.get(url, headers=headers)

    data = response.json()

    identified_programs = {}

    for account in data['programs']:
        program_id = account.get('programId')
        program_id = program_id.lower()
        name = account.get('name')
        identified_programs[program_id] = name

    return identified_programs

In [12]:
vybe_programs_map = get_vybe_identified_programs()
vybe_addresses_map = get_vybe_identified_accounts()

In [13]:
# # Normalize mapping keys
# vybe_programs_map = {k.lower(): v for k, v in vybe_programs_map.items()}
# vybe_addresses_map = {k.lower(): v for k, v in vybe_addresses_map.items()}

# # Apply human-readable labels to tx-level DataFrame
# test_wallet_timeseries['sender_name'] = test_wallet_timeseries['sender'].astype(str).str.lower().map(
#     lambda addr: vybe_addresses_map.get(addr, 'Unknown Address'))

# test_wallet_timeseries['receiver_name'] = test_wallet_timeseries['receiver'].astype(str).str.lower().map(
#     lambda addr: vybe_addresses_map.get(addr, 'Unknown Address'))

# test_wallet_timeseries['counterparty_name'] = test_wallet_timeseries['counterparty'].astype(str).str.lower().map(
#     lambda addr: vybe_addresses_map.get(addr, 'Unknown Address'))

# test_wallet_timeseries['program_name'] = test_wallet_timeseries['program_id'].astype(str).str.lower().map(
#     lambda pid: vybe_programs_map.get(pid, 'Unknown Program'))

# test_wallet_timeseries['wallet_entity_label'] = vybe_addresses_map.get(test_address.lower(), 'Unknown Entity')

In [14]:
# 'sender_name', 'receiver_name', 'counterparty_name', 'program_name',
#        'wallet_entity_label'

In [15]:
list(vybe_programs_map.values())

['Star Atlas SAGE',
 'Cardinal Token Manager',
 'Pyth Staking',
 'Star Atlas - PROFILE ACTION',
 'Hadeswap',
 'Helium Voter Stake Registry',
 'HXRO - Risk',
 'Citrus',
 'MPL Auction House',
 'Zeta Markets',
 'Star Atlas - CRAFTING',
 'Marginfi',
 'Orca-Openbook Bot',
 'Stabble Stable Swap',
 'Monaco Protocol',
 'Star Atlas Proxy Rewarder',
 'Raydium AMM Route',
 'Cykura',
 'Titan Dex ',
 'Streamflow MultiSig',
 'Parrot Protocol',
 'Socean Program',
 'Dradex',
 'Solend',
 'GooseFx v1',
 'Jet Protocol Rewards',
 'Zeta Matching Engine',
 'Crema Finance',
 'Tensor Compressed',
 'Taiyo Launchpad',
 'MPL Candy Guard',
 'Jupiter Swap V3',
 'Meta DAO Futarchy',
 'GooseFX Controller',
 'Marinade - Directed Stake',
 'Hub3.ee',
 'Stabble Weighted Swap',
 'Star Atlas FACTION ENLISTMENT',
 'Raydium CLMM Program',
 'Aldrin AMM V1',
 'Fluxbeam',
 'Jupiter v5',
 "HXRO - Fee's",
 ' Squads Program ID V4',
 'Meteora DLMM Program',
 'MPL Bubblegum',
 'Streamflow',
 'Orca Whirlpool',
 'Lifinity Swap V2',
 

In [16]:
list(vybe_addresses_map.values())

['@solkcrow',
 'Openbook CATO-USDC',
 'Raydium CWAR-USDC',
 'Orca SOL-FISHY',
 'Raydium SBONK-SOL',
 'Magic Eden V2 Authority',
 'Orca SOL-J-SOL',
 'Meteora DLMM HNT-USDC',
 'Raydium mSOL-whETH',
 'verybrickedup',
 'Raydium KIN-USDC',
 'Orca SOL-USDC',
 'Raydium SOL-ZBC',
 'Orca PORT-BONK',
 'Raydium KING-USDC',
 'Orca SOL-DUST',
 'Raydium mSOL-USDT',
 'Orca SOL-PHY',
 'Openbook KART-SOL',
 'Orca SOL-ATLAS',
 'Orca GUAC-USDC',
 'Orca USDC-UPS',
 'Orca UXD-NANA',
 'Openbook MeMe-USDC',
 'Openbook BULL-SOL',
 'Openbook LMDA-SOL',
 'Raydium RBL-USDC',
 'Raydium BBY-USDC',
 'Openbook FRONK-HADES',
 'MarginFi Bonk Asset Pool',
 'Openbook WOOF-USDC',
 'Raydium SOL-SLO',
 'Orca USDC-ISC',
 'Openbook MOLA-USDC',
 'Raydium USDC-JAM',
 'Orca ORCA-USDC',
 'Raydium SOL-WSSI',
 'Openbook AFSeX-SOL',
 'Orca ORCA-stSOL',
 'Raydium LAMDA-USDC',
 'Raydium SOL-BLAKE',
 'Orca SOL-stSOL',
 'Raydium MILK-USDC',
 'Coinbase Exchange 1',
 'Raydium SOL-RAY',
 'Raydium 7SBw57eejb7gob2GM7EWh1xqxS1hBvwWqRmg8aLjWo

# Helius web3 Names

I think this endpoint is deprecated

In [17]:
from helius import NameAPI
HELIUS_API_KEY = os.getenv('HELIUS_API_KEY')

name_api  = NameAPI(HELIUS_API_KEY)


In [18]:
# name = name_api.get_address_name(test_address)
# print(name)

# Metasleuth Labels

In [19]:
METASLEUTH_API_KEY = os.getenv('METASLEUTH_API_KEY')

In [20]:
# import requests

# response = requests.get(
#     "https://aml.blocksec.com/address-label/api/v3/chain-list",
#     headers={"API-KEY":METASLEUTH_API_KEY},
# )

# data = response.json()

In [21]:
# print(data['data'])

In [22]:
def get_solana_chainid(data):
    for d in data['data']:
        chain_id = d.get('chain_id')
        chain_name = d.get('chain_name')

        if chain_name.lower() == 'solana':
            return {chain_id:chain_name}

In [23]:
# solana_metadata = get_solana_chainid(data)


In [24]:
# list(solana_metadata.keys())[0]

In [25]:
filtered_addresses = unique_addresses[:100] # current API key only allows 100 addresses per day, this way we get latest 100
len(filtered_addresses)

100

In [26]:
def chunked(iterable, size):
    """Yield successive `size`-sized chunks from iterable."""
    for i in range(0, len(iterable), size):
        yield iterable[i:i + size]

def fetch_address_labels(unique_addresses, chain_id, api_key, delay=0.2):
    url = "https://aml.blocksec.com/address-label/api/v3/batch-labels"
    headers = {
        "API-KEY": api_key,
        "Content-Type": "application/json"
    }
    results = []

    for chunk in chunked(unique_addresses, 100):
        response = requests.post(
            url,
            headers=headers,
            json={
                "chain_id": chain_id,
                "addresses": chunk
            }
        )
        data = response.json()
        print(f'data: {data}')
        if data.get("code") == 200000:
            results.extend(data.get("data", []))
        else:
            print(f"[Error] Batch failed: {data}")
        time.sleep(delay)  # optional delay to avoid rate limits

    return results

In [27]:
filtered_addresses

['G3xN64JxT7Mt8tGqzrXRWd2PcHj5LwT9DpVNuAYjDZJu',
 '4NFwLsQLdezXpSgSjHqJWJygcH3o5xxr1TcMZ98joak9',
 '2DNxPzaQkmAxsEzWd4NM1soPr6q6CuB3voPr8f5xNiQc',
 '5FRUZAmmkE3A71ejb1XQm3vVAazkD8F3jz55Z8EcGG3i',
 '5Q544fKrFoe6tsEbD7S8EmxGTJYAKtTVhAW5Q5pge4j1',
 '5PXvdatun2zBL7aSi2qrQusUSmGrhspHGXkFKr6Wy7qc',
 '5wckPMwQDNxKRDjJ1ZiKRgtBogvLtaHKoK5AGPr7V89D',
 'ELJk5HtKjjrLCxumui5Ahcq3XtyB4tKYr7NhLqQrvy3e',
 '7E2tFVCmm4PDhxacrQ8i91aZyg8cmTXpSwJ8EybHWXwz',
 '4BbjpFeB6bT7UD9gQTQh9BcWgEsM46B9dzC6yGTjtaoX',
 'EoYr3pK5iqG4bZJZTKJyBnpaPSU77EHfv3T1iVMFoVWb',
 '3KZTzvQEPx3uvnEBywF3pVy2PF6tq7KXE2auRWfP27pN',
 '8YAQXVFSxFdwHKh9PMnPJtKSU24xLCbpGyU1fsToLotv',
 '242NBZcHP4tEGhQWr2vcRqUcD9kuNYZZNycY2MxydBmi',
 '2AEWSvUds1wsufnsDPCXjFsJCMJH5SNNm7fSF4kxys9a',
 '8JLsZXzWGGuQeyH3jLdqaK9LMgxWZNxGeGPCwBUWZuyq',
 '2EEw1A49utRqUsnYRYWtnpz2UQ7n7sJn8EEpdN2nKqWQ',
 'FLiPggWYQyKVTULFWMQjAk26JfK5XRCajfyTmD5weaZ7',
 'EJ1b6HQNk13UQW4rtiRCGMzPTLTnMSDQTwuwf7h73FaD',
 '2fnfKetCS7CyTr9MwALByXCjA9bYU4xbySUBtK3ERvA7',
 '6zZTTZ38EDm2ozbHXi

In [28]:
# solana_labels = fetch_address_labels(
#     unique_addresses=filtered_addresses,
#     chain_id=-3,  # SOLANA
#     api_key=METASLEUTH_API_KEY
# )


In [29]:
# with open('../data/raw/metasleuth_labels.json', 'w') as f:
#             json.dump(solana_labels, f)

In [30]:
with open('../data/raw/metasleuth_labels.json') as f:
    solana_labels = json.load(f)

In [31]:
len(solana_labels)

16

In [32]:
solana_labels[0].keys()

dict_keys(['chain_id', 'address', 'main_entity', 'main_entity_info', 'comp_entities', 'attributes', 'name_tag'])

In [33]:
entities_dict = {}

for l in solana_labels:
    entity = l['main_entity']
    address = l['address']

    entities_dict[address] = entity

In [34]:
list(entities_dict.values())

['Orca',
 'Coinbase Exchange',
 'MEXC',
 'Jito',
 'Jupiter',
 'Jupiter',
 'Jupiter',
 'Orca',
 'Orca',
 'Orca',
 'Jupiter',
 'Phantom',
 'Jupiter',
 'Jupiter',
 'Coinbase Exchange',
 'Jupiter']

# SolanaScan

In [35]:
SOLSCAN_API_KEY = os.getenv('SOLSCAN_API_KEY')

In [36]:
# url = "https://pro-api.solscan.io/v2.0/transaction/detail?tx=65mbqVyFo9kGav1zdCTUjdYtVZkRb82m2E9ZqedDwWdKDiPKuxNKmMsVHFWnBxMin2SncasNB8FGmMM7ANRFRtLn"    

# headers = {"token":SOLSCAN_API_KEY}

# response = requests.get(url, headers=headers)

# print(response.text)

# data = response.json()


In [37]:
# data['data'].keys()

# Combining all Label Maps to DF

In [38]:
entities_dict

{'5snaYowgJDfuM1LPbTNUYHbgkKHtVVnzHiiLDWUV2hh8': 'Orca',
 '2AQdpHJ2JpcEgPiATUXjQxA8QmafFegfQwSLWSprPicm': 'Coinbase Exchange',
 'ASTyfSima4LLAdDgoFGkgqoKowG1LZFDr9fAQrg7iaJZ': 'MEXC',
 'ADuUkR4vqLUMWXxW9gh6D6L8pMSawimctcNZ5pGwDcEt': 'Jito',
 'CapuXNQoDviLvU1PxFiizLgPNQCxrsag1uMeyk6zLVps': 'Jupiter',
 '45ruCyfdRkWpRNGEqWzjCiXRHkZs8WXCLQ67Pnpye7Hp': 'Jupiter',
 '4xDsmeTWPNjgSVSS1VTfzFq3iHZhp77ffPkAmkZkdu71': 'Jupiter',
 'GUWM1arUyDnkMGCHvJu3yt1qomJ988utqC3dFN2AUCDT': 'Orca',
 '7qbRF6YsyGuLUVs6Y1q64bdVrfe4ZcUUz1JRdoVNUJnm': 'Orca',
 'HQcY5n2zP6rW74fyFEhWeBd3LnJpBcZechkvJpmdb8cx': 'Orca',
 '2MFoS3MPtvyQ4Wh4M9pdfPjz6UhVoNbFbGJAskCPCj3h': 'Jupiter',
 'tzvXws1qhmfdPkPcprezULCDQPAJqzPhbZ3SMrqRPNE': 'Phantom',
 '9nnLbotNTcUhvbrsA6Mdkx45Sm82G35zo28AqUvjExn8': 'Jupiter',
 '6LXutJvKUw8Q5ue2gCgKHQdAN4suWW8awzFVC6XCguFx': 'Jupiter',
 'H8sMJSCQxfKiFTCfDR3DUMLPwcRbM61LGFJ8N4dK3WjS': 'Coinbase Exchange',
 'BQ72nSv9f3PRyRKCBnHLVrerrv37CYTHm5h3s9VSGQDV': 'Jupiter'}

In [39]:
vybe_programs_map

{'sageqqfewepdhh6hmdcmwy7yjhppykldnrxkb3ki8e6': 'Star Atlas SAGE',
 'mgr99qfmybytqgpwmnqunv7vblmwwxdsrhufv8jf3jm': 'Cardinal Token Manager',
 'pyts9tjg1qyazypk7n8rw8gfw9suaqqyymhjq4e7jcq': 'Pyth Staking',
 'pfacsruobdmvfmkq1bazwj27t6d2gjhschb1vcfnrmq': 'Star Atlas - PROFILE ACTION',
 'hadek9dlv9ea7ya5kctqsvsvrzejc3jgd5a9y3cnbvu': 'Hadeswap',
 'hvsrnc3nkbcryqds2docyhz9ypkevzdsjqg6rvtk1s8': 'Helium Voter Stake Registry',
 '92wdgeqyidkrcbfhobtg8hxmj932xwerckacigsw3umr': 'HXRO - Risk',
 'jcfrapv7852esrwjjgry2mysumydxzgvvhrmlmexvmvp': 'Citrus',
 'hauss13jsjafwwwgqztuqrmwyvyxn9eqpqmwv1pbbmk': 'MPL Auction House',
 'zetaxsqbrek56dhigxrn75yj2nhu3ayunxvhxpkf3ad': 'Zeta Markets',
 'craftf1egzeopfj1rpatsqg1f6hhrrbaf4gro9hdszjr': 'Star Atlas - CRAFTING',
 'mfv2hwf31z9kbca1snepyctwafyhdvnv7fznsebvaca': 'Marginfi',
 '6f3n8a6fccrb8hxnpv3t2ludfmjk6vby7miqcjpuuo6a': 'Orca-Openbook Bot',
 'swapnyd8xiqwj6ianp9snpu4bruqfxadzvhebnaxjjz': 'Stabble Stable Swap',
 'monacouxktui6vksqwalyxmxksievfnwecyxtgkbcih'

In [40]:
# Normalize mapping keys
vybe_programs_map = {k.lower(): v for k, v in vybe_programs_map.items()}
vybe_addresses_map = {k.lower(): v for k, v in vybe_addresses_map.items()}
entities_dict = {k.lower(): v for k, v in entities_dict.items()} # metasleuth
flipside_labels_dict = {k.lower(): v for k, v in flipside_labels_dict.items()} # flipside

In [41]:
combined_address_label_map = {
    **vybe_addresses_map,
    **entities_dict,
    **flipside_labels_dict
}
len(combined_address_label_map.keys())

105967

In [42]:


# Apply human-readable labels to tx-level DataFrame
test_wallet_timeseries['sender_name'] = test_wallet_timeseries['sender'].astype(str).str.lower().map(
    lambda addr: combined_address_label_map.get(addr, 'Unknown Address'))

test_wallet_timeseries['receiver_name'] = test_wallet_timeseries['receiver'].astype(str).str.lower().map(
    lambda addr: combined_address_label_map.get(addr, 'Unknown Address'))

test_wallet_timeseries['counterparty_name'] = test_wallet_timeseries['counterparty'].astype(str).str.lower().map(
    lambda addr: combined_address_label_map.get(addr, 'Unknown Address'))

test_wallet_timeseries['program_name'] = test_wallet_timeseries['program_id'].astype(str).str.lower().map(
    lambda pid: vybe_programs_map.get(pid, 'Unknown Program'))



In [43]:
test_wallet_timeseries['wallet_entity_label'] = combined_address_label_map.get(test_address.lower(), 'Unknown Entity')

In [44]:
test_wallet_timeseries[test_wallet_timeseries['receiver_name']!='Unknown Address']['receiver_name'].unique()

array(['Orca', 'Jupiter', 'Jupiter Aggregator Authority',
       'tip payment account', 'Orca SOL-USDC', 'Phantom', 'Orca SOL-bSOL',
       'kamino fee wallet', 'Kamino Reserve 1', 'Orca SOL-stSOL',
       'Raydium AMM Authority'], dtype=object)

In [45]:
test_wallet_timeseries[test_wallet_timeseries['sender_name']!='Unknown Address']['sender_name'].unique()

array(['Orca', 'Jupiter', 'MEXC', 'Jupiter Aggregator Authority',
       'Orca SOL-USDC', 'Coinbase Exchange', 'Orca SOL-bSOL',
       'Kamino Reserve 1', 'SolBlaze Airdrop Wallet',
       'coinbase hot wallet', 'Orca SOL-stSOL', 'Raydium AMM Authority'],
      dtype=object)

In [46]:
test_wallet_timeseries[test_wallet_timeseries['program_name']!='Unknown Program']['program_name'].unique()

array(['Jupiter Swap V6', 'Allbridge Core', 'MPL Bubblegum',
       'PENGU Airdrop Claiming', 'Jupiter Swap V2', 'Parcl V3',
       'Kamino Program', 'Kamino Lend'], dtype=object)

# Category Labeling

In [47]:
combined_address_label_map

{'aprnqn2hkbcn7w2wwit2fekvukjp9lugryae1a9hdz1': '@solkcrow',
 '12qcuqcsmz3yx2b6268hhtrtbciwrx4hjjaagtrvn7nz': 'Openbook CATO-USDC',
 '13ucpybnakxhgvd2ddvb7o2uwxuf9gqpfkvjmvgky6uj': 'Raydium CWAR-USDC',
 '13ztudny8hvzckbk7xwrv5jt5fd9ks7bgdxp1ml1gd3h': 'Orca SOL-FISHY',
 '14ghvv3isps8cndf6pazgzb1tqs2l1yvjqanvezsiszu': 'Raydium SBONK-SOL',
 '1bwutmtvypwdtmw9abtks4ssr8no61spgavw1x6ndix': 'Magic Eden V2 Authority',
 '1fdbob8pszavbzcwnpebhjhvurfa7bge1bqk2jwmys4': 'Orca SOL-J-SOL',
 '1koyvnej5gwxz6v3re8xwxdhrephf4vnyrngv4bhrqk': 'meteora: hnt / usdc',
 '212sadyxtdwdn4v127v3k4oz2kdjvcrn7qpdsobnpahv': 'Raydium mSOL-whETH',
 '215nhcahjqqggwpqsjq7zr26etbjjtvdw74nlzwegqjp': 'verybrickedup',
 '21s8fctrhyontyaazzmjebjwyvraddzw8op74yn4wbya': 'Raydium KIN-USDC',
 '21gtfxanhudjjgzjdktxctgfkt8teixx6pn1ceg9k1uw': 'Orca SOL-USDC',
 '21nx2rm9zzjjofu3xrxlee1dq9jegj789exgv4teyjb3': 'Raydium SOL-ZBC',
 '2243xchapybqw4yazgve1bncc2ksnvr9wbctwjgqobn8': 'Orca PORT-BONK',
 '22dbujl2ztsu49ujyqus3shnftphnvkttguvfwjb

In [48]:
df = pd.DataFrame(list(combined_address_label_map.items()), columns=['address', 'entity'])

# Preview
print(df.head())

                                        address              entity
0   aprnqn2hkbcn7w2wwit2fekvukjp9lugryae1a9hdz1           @solkcrow
1  12qcuqcsmz3yx2b6268hhtrtbciwrx4hjjaagtrvn7nz  Openbook CATO-USDC
2  13ucpybnakxhgvd2ddvb7o2uwxuf9gqpfkvjmvgky6uj   Raydium CWAR-USDC
3  13ztudny8hvzckbk7xwrv5jt5fd9ks7bgdxp1ml1gd3h      Orca SOL-FISHY
4  14ghvv3isps8cndf6pazgzb1tqs2l1yvjqanvezsiszu   Raydium SBONK-SOL


In [49]:
test_wallet_timeseries[test_wallet_timeseries['token_address']=='COMPRESSED_NFT']

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,...,TOKEN_NAME,symbol,day,price,token_amount_usd,sender_name,receiver_name,counterparty_name,program_name,wallet_entity_label
125,2025-02-03 17:43:37,3TPZSpTA5QKz5AQ3g7yX2ELnQHWYRGiuKu9T8GgVjBfSvZ...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,318251172.0,COMPRESSED_NFT,1.0,received,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,...,NaN,AB Open #001,2025-02-03,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity
127,2025-02-03 17:43:22,35ie9YJLNVcMGA751C2aN2JHoN7YidMmgntEw2pg3QLEzw...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,318251136.0,COMPRESSED_NFT,1.0,received,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,...,NaN,AB Open #001,2025-02-03,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity
225,2024-12-31 01:27:48,5PKmHyHf9C6ASTt2HWqXy3iMuRLofE3i6HAB88kYEHUuFC...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310883171.0,COMPRESSED_NFT,1.0,received,59suVJh2jLvADqgxv3cUmNMm9g3hjXb5M6vu29jV27Tt,...,NaN,Grant #3,2024-12-31,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity
241,2024-12-30 17:20:28,5zMbFS9efMXYjwN28zgzDGo5j8VNiPikgc8YAv1EfHUBtX...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310811606.0,COMPRESSED_NFT,1.0,received,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,...,NaN,🌟 Gift #314,2024-12-30,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity
242,2024-12-30 17:20:28,5zMbFS9efMXYjwN28zgzDGo5j8VNiPikgc8YAv1EfHUBtX...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310811606.0,COMPRESSED_NFT,1.0,received,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,...,NaN,🌟 Gift #320,2024-12-30,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity
250,2024-12-29 15:35:01,2DVEywsq4Qt2KgkmZMuP3FrB1UZVPN6Rvi9rHUFYmoBEzz...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310585694.0,COMPRESSED_NFT,1.0,received,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,...,NaN,☄️JUPITER☄️ AIRDROP,2024-12-29,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity
252,2024-12-29 15:34:31,4Fnj33Jq5xe7MZcdVa15su8VKMXXW6Jc2y6d4FDeoVEQj5...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310585620.0,COMPRESSED_NFT,1.0,received,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,...,NaN,☄️JUPITER☄️ AIRDROP,2024-12-29,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity
257,2024-12-23 16:10:06,HB6rZmSiSNJoDTz45cwdmRDMCnJZKdLod2v9shroCBAdwf...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,309332138.0,COMPRESSED_NFT,1.0,received,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,...,NaN,BOX#8,2024-12-23,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity
275,2024-12-21 22:38:42,4gtgKHj7aum7JBcb4Bm9Norw3216dQkzHKtxBFznskoB48...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,308967649.0,COMPRESSED_NFT,1.0,received,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,...,NaN,5000 TO 50000 #1,2024-12-21,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity
279,2024-12-21 22:37:42,a8aKKJugCLBf1kC8brAakSkZAEHgztfiqgCth9ghNY33eH...,COMPRESSED_NFT_MINT,BUBBLEGUM,success,308967503.0,COMPRESSED_NFT,1.0,received,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,...,NaN,5000 TO 50000 #68,2024-12-21,NaN,NaN,Unknown Address,Unknown Address,Unknown Address,MPL Bubblegum,Unknown Entity


In [50]:
# Rule-based categorization function
def classify_category(entity):
    entity = entity.lower()

    # Exchanges
    if any(keyword in entity for keyword in ['openbook', 'raydium', 'orca', 'jupiter', 'meteora', 'coinbase']):
        return 'Exchange'

    # NFT Traders or Platforms
    if any(keyword in entity for keyword in ['magic eden', 'nft', 'digitaleyes', 'tensor']):
        return 'NFT Trader'

    # Rug pulls (hardcoded suspicious names or memes)
    if any(keyword in entity for keyword in ['rug', 'bricked', 'scam', 'meme']):
        return 'Rug Pull'
    
    if any(keyword in entity for keyword in ['allbridge']):
        return 'Bridge'
    
    if any(keyword in entity for keyword in ['airdrop','pengu']):
        return 'Airdrop'

    # DeFi protocols
    if any(keyword in entity for keyword in ['uxd', 'usdh', 'softt', 'vault', 'solend', 'lending','lend','kamino','parcl']):
        return 'DeFi Protocol'

    # Default fallback
    return 'Other'

In [51]:
test_wallet_timeseries['sender_category'] = test_wallet_timeseries['sender_name'].apply(classify_category)
test_wallet_timeseries['receiver_category'] = test_wallet_timeseries['receiver_name'].apply(classify_category)
test_wallet_timeseries['program_category'] = test_wallet_timeseries['program_name'].apply(classify_category)

In [52]:
test_wallet_timeseries[~test_wallet_timeseries['sender_category'].isin(['Other','Exchange'])]

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,...,price,token_amount_usd,sender_name,receiver_name,counterparty_name,program_name,wallet_entity_label,sender_category,receiver_category,program_category
744,2024-01-27 20:00:42,3kTQ7DYxvUoknsnHkHd4wMGCvXgq5CdXM6ZPUUnfJc1tGB...,WITHDRAW_OBLIGATION_COLLATERAL_AND_REDEEM_RESE...,KAMINO_LEND,success,244500759.0,9uvxPXzVYWe8qb7S4gUBqsvASmRMVWo5GSAcMmdZ1LQr,21.284506,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,NaN,NaN,Kamino Reserve 1,Unknown Address,Kamino Reserve 1,Kamino Lend,Unknown Entity,DeFi Protocol,Other,DeFi Protocol
745,2024-01-27 20:00:42,3kTQ7DYxvUoknsnHkHd4wMGCvXgq5CdXM6ZPUUnfJc1tGB...,WITHDRAW_OBLIGATION_COLLATERAL_AND_REDEEM_RESE...,KAMINO_LEND,success,244500759.0,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,21.284506,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,NaN,NaN,Kamino Reserve 1,Unknown Address,Kamino Reserve 1,Kamino Lend,Unknown Entity,DeFi Protocol,Other,DeFi Protocol
796,2024-01-27 19:54:37,2JGMFWYbCcR51SgXUPejvvpRRHAs1xyxPgTz96DrrXNNmb...,WITHDRAW_OBLIGATION_COLLATERAL_AND_REDEEM_RESE...,KAMINO_LEND,success,244499879.0,9uvxPXzVYWe8qb7S4gUBqsvASmRMVWo5GSAcMmdZ1LQr,38.261386,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,NaN,NaN,Kamino Reserve 1,Unknown Address,Kamino Reserve 1,Kamino Lend,Unknown Entity,DeFi Protocol,Other,DeFi Protocol
797,2024-01-27 19:54:37,2JGMFWYbCcR51SgXUPejvvpRRHAs1xyxPgTz96DrrXNNmb...,WITHDRAW_OBLIGATION_COLLATERAL_AND_REDEEM_RESE...,KAMINO_LEND,success,244499879.0,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,38.261386,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,NaN,NaN,Kamino Reserve 1,Unknown Address,Kamino Reserve 1,Kamino Lend,Unknown Entity,DeFi Protocol,Other,DeFi Protocol
818,2024-01-27 19:51:51,t1EgxX8diQh5Ak4ptKnY6ajRu96dwE2R8yYJraQsSJ4Cxy...,WITHDRAW_OBLIGATION_COLLATERAL_AND_REDEEM_RESE...,KAMINO_LEND,success,244499483.0,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,2.880316,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,NaN,NaN,Kamino Reserve 1,Unknown Address,Kamino Reserve 1,Kamino Lend,Unknown Entity,DeFi Protocol,Other,DeFi Protocol
817,2024-01-27 19:51:51,t1EgxX8diQh5Ak4ptKnY6ajRu96dwE2R8yYJraQsSJ4Cxy...,WITHDRAW_OBLIGATION_COLLATERAL_AND_REDEEM_RESE...,KAMINO_LEND,success,244499483.0,9uvxPXzVYWe8qb7S4gUBqsvASmRMVWo5GSAcMmdZ1LQr,2.880316,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,NaN,NaN,Kamino Reserve 1,Unknown Address,Kamino Reserve 1,Kamino Lend,Unknown Entity,DeFi Protocol,Other,DeFi Protocol
821,2024-01-27 19:47:10,DEYMhgGRURmjVATdnhMQAUsZFxHzRXftxPS3E5BGteS7VJ...,WITHDRAW_OBLIGATION_COLLATERAL_AND_REDEEM_RESE...,KAMINO_LEND,success,244498809.0,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,10.000000,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,NaN,NaN,Kamino Reserve 1,Unknown Address,Kamino Reserve 1,Kamino Lend,Unknown Entity,DeFi Protocol,Other,DeFi Protocol
820,2024-01-27 19:47:10,DEYMhgGRURmjVATdnhMQAUsZFxHzRXftxPS3E5BGteS7VJ...,WITHDRAW_OBLIGATION_COLLATERAL_AND_REDEEM_RESE...,KAMINO_LEND,success,244498809.0,9uvxPXzVYWe8qb7S4gUBqsvASmRMVWo5GSAcMmdZ1LQr,10.000000,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,NaN,NaN,Kamino Reserve 1,Unknown Address,Kamino Reserve 1,Kamino Lend,Unknown Entity,DeFi Protocol,Other,DeFi Protocol
833,2024-01-27 19:44:21,2GZgJBTkg7W7y7xPiw7c8vUhzFwe7w6ZuKucQLXgbppo4R...,BORROW_OBLIGATION_LIQUIDITY,KAMINO_LEND,success,244498408.0,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,0.170000,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,107.466966,18.269384,Kamino Reserve 1,Unknown Address,Kamino Reserve 1,Kamino Lend,Unknown Entity,DeFi Protocol,Other,DeFi Protocol
857,2024-01-11 22:48:59,dDnHST65GUQcUAPYdTK8wMYrpWW1ScnMAfomurL7WKM9d3...,BORROW_OBLIGATION_LIQUIDITY,KAMINO_LEND,success,241258967.0,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.240000,received,9DrvZvyWh1HuAoZxvYWMvkf2XCzryCpGgHqrMjyDWpmo,...,111.591073,26.781858,

In [53]:
test_wallet_timeseries[~test_wallet_timeseries['receiver_category'].isin(['Other','Exchange'])]

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,...,price,token_amount_usd,sender_name,receiver_name,counterparty_name,program_name,wallet_entity_label,sender_category,receiver_category,program_category
740,2024-01-27 20:02:00,AG9Wy4nnG7vgMgVwmBmiKUdfcGk9Sigh9XHBgUMq9LCfL8...,UNKNOWN,ORCA,success,244500947.0,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.000652,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,102.083897,0.066509,Unknown Address,kamino fee wallet,kamino fee wallet,Kamino Program,Unknown Entity,Other,DeFi Protocol,DeFi Protocol
739,2024-01-27 20:02:00,AG9Wy4nnG7vgMgVwmBmiKUdfcGk9Sigh9XHBgUMq9LCfL8...,UNKNOWN,ORCA,success,244500947.0,So11111111111111111111111111111111111111112,0.000268,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,92.352398,0.024791,Unknown Address,kamino fee wallet,kamino fee wallet,Kamino Program,Unknown Entity,Other,DeFi Protocol,DeFi Protocol
747,2024-01-27 20:00:26,3VUEpSAXdBJ63kPnFVbqp1Wne6WT3GXaUuGZwHPRpByHgB...,REPAY_OBLIGATION_LIQUIDITY,KAMINO_LEND,success,244500721.0,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,0.170000,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,107.466966,18.269387,Unknown Address,Kamino Reserve 1,Kamino Reserve 1,Kamino Lend,Unknown Entity,Other,DeFi Protocol,DeFi Protocol
754,2024-01-27 19:57:09,FQAFj74spD88csA6Xgg6WTRFobQ7DgvkeDqTBSkRqTScaK...,REPAY_OBLIGATION_LIQUIDITY,KAMINO_LEND,success,244500253.0,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.143122,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,102.083897,14.610477,Unknown Address,Kamino Reserve 1,Kamino Reserve 1,Kamino Lend,Unknown Entity,Other,DeFi Protocol,DeFi Protocol
759,2024-01-27 19:56:38,5XbLMj697ixEAzt6aZMD3fXx26cKYUwHmA2PmqMPYAcDry...,UNKNOWN,ORCA,success,244500178.0,So11111111111111111111111111111111111111112,0.000483,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,92.352398,0.044591,Unknown Address,kamino fee wallet,kamino fee wallet,Unknown Program,Unknown Entity,Other,DeFi Protocol,Other
760,2024-01-27 19:56:38,5XbLMj697ixEAzt6aZMD3fXx26cKYUwHmA2PmqMPYAcDry...,UNKNOWN,ORCA,success,244500178.0,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.001171,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,102.083897,0.119529,Unknown Address,kamino fee wallet,kamino fee wallet,Unknown Program,Unknown Entity,Other,DeFi Protocol,Other
768,2024-01-27 19:56:17,61KTx2PdE9RkeBsDhoCkdThobprACtKJohDXmGLh5nhG3o...,UNKNOWN,ORCA,failed,244500126.0,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.001171,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,102.083897,0.119526,Unknown Address,kamino fee wallet,kamino fee wallet,Kamino Program,Unknown Entity,Other,DeFi Protocol,DeFi Protocol
767,2024-01-27 19:56:17,61KTx2PdE9RkeBsDhoCkdThobprACtKJohDXmGLh5nhG3o...,UNKNOWN,ORCA,failed,244500126.0,So11111111111111111111111111111111111111112,0.000483,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,92.352398,0.044594,Unknown Address,kamino fee wallet,kamino fee wallet,Kamino Program,Unknown Entity,Other,DeFi Protocol,DeFi Protocol
776,2024-01-27 19:55:27,3ej3AQV8BiJd5ipwYvcBFLLc1iSg4UZX74H3h6VTQWW9VB...,UNKNOWN,ORCA,failed,244500002.0,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.001120,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,102.083897,0.114324,Unknown Address,kamino fee wallet,kamino fee wallet,Kamino Program,Unknown Entity,Other,DeFi Protocol,DeFi Protocol
775,2024-01-27 19:55:27,3ej3AQV8BiJd5ipwYvcBFLLc1iSg4UZX74H3h6VTQWW9VB...,UNKNOWN,ORCA,failed,244500002.0,So11111111111111111111111111111111111111112,0.000462,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,92.352398,0.042653,Unknown Address,kamino fee wallet,kamino fee wallet,Kamino Program,Unknown Entity,Other,DeFi Protocol,DeFi Protocol


In [54]:
test_wallet_timeseries[~test_wallet_timeseries['program_category'].isin(['Other','Exchange','Bridge'])]

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,...,price,token_amount_usd,sender_name,receiver_name,counterparty_name,program_name,wallet_entity_label,sender_category,receiver_category,program_category
267,2024-12-23 16:06:42,637N8mjfeSiDpovn2oPCjnCXbj6RRdq12jZQR54G5RfU28...,UNKNOWN,UNKNOWN,success,309331660.0,So11111111111111111111111111111111111111111,0.002039,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,183.460000,0.374126,Unknown Address,Unknown Address,Unknown Address,PENGU Airdrop Claiming,Unknown Entity,Other,Other,Airdrop
268,2024-12-23 16:06:42,637N8mjfeSiDpovn2oPCjnCXbj6RRdq12jZQR54G5RfU28...,UNKNOWN,UNKNOWN,success,309331660.0,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,3490.000000,received,2EEw1A49utRqUsnYRYWtnpz2UQ7n7sJn8EEpdN2nKqWQ,...,0.027035,94.352822,Unknown Address,Unknown Address,Unknown Address,PENGU Airdrop Claiming,Unknown Entity,Other,Other,Airdrop
715,2024-01-28 19:52:53,3DpQT9VumSzjPgrH65pv67c4uLjuWnMoGjp2fzGsNDReXx...,UNKNOWN,UNKNOWN,success,244706943.0,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,19.144865,received,82dGS7Jt4Km8ZgwZVRsJ2V6vPXEhVdgDaMP7cqPGG1TW,...,1.000156,19.147853,Unknown Address,Unknown Address,Unknown Address,Parcl V3,Unknown Entity,Other,Other,DeFi Protocol
742,2024-01-27 20:02:00,AG9Wy4nnG7vgMgVwmBmiKUdfcGk9Sigh9XHBgUMq9LCfL8...,UNKNOWN,ORCA,success,244500947.0,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.000001,received,7qfrHwfk3PKF7UMZFUuSHzBzgv4pnAUcszkfRfXsgwzV,...,102.083897,0.000150,Unknown Address,Unknown Address,Unknown Address,Kamino Program,Unknown Entity,Other,Other,DeFi Protocol
740,2024-01-27 20:02:00,AG9Wy4nnG7vgMgVwmBmiKUdfcGk9Sigh9XHBgUMq9LCfL8...,UNKNOWN,ORCA,success,244500947.0,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.000652,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,102.083897,0.066509,Unknown Address,kamino fee wallet,kamino fee wallet,Kamino Program,Unknown Entity,Other,DeFi Protocol,DeFi Protocol
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,2023-11-19 23:48:54,5BUvtLsszmv38C3LJbpqqyjcyxg6UT4AkH8481MLfcJW9U...,UNKNOWN,UNKNOWN,success,231056031.0,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,15.520000,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,0.999970,15.519530,Unknown Address,Unknown Address,Unknown Address,Parcl V3,Unknown Entity,Other,Other,DeFi Protocol
957,2023-11-19 23:48:02,BTihkadXbWfxFyPZBiDLCD7aw23qAB4RmiUi3Vct4hjyaJ...,UNKNOWN,UNKNOWN,success,231055906.0,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,15.000000,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,0.999970,14.999546,Unknown Address,Unknown Address,Unknown Address,Parcl V3,Unknown Entity,Other,Other,DeFi Protocol
956,2023-11-19 23:48:02,BTihkadXbWfxFyPZBiDLCD7aw23qAB4RmiUi3Vct4hjyaJ...,UNKNOWN,UNKNOWN,success,231055906.0,So11111111111111111111111111111111111111111,0.001837,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,59.621687,0.109551,Unknown Address,Unknown Address,Unknown Address,Parcl V3,Unknown Entity,Other,Other,DeFi Protocol
969,2023-11-14 22:30:26,2c4Tsa8zfm19HDXSRQSoVr1je22bEVCthU3YGjFhXreMLE...,UNKNOWN,UNKNOWN,success,230045651.0,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,13.162149,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,...,0.999951,13.161499,Unknown Address,Unknown Address,Unknown Address,Parcl V3,Unknown Entity,Other,Other,DeFi Protocol


In [55]:
test_wallet_timeseries.to_csv('../data/processed/test_acc_timeseries_labeled.csv', index=False)

In [56]:
test_wallet_timeseries.head()

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,...,price,token_amount_usd,sender_name,receiver_name,counterparty_name,program_name,wallet_entity_label,sender_category,receiver_category,program_category
0,2025-04-06 12:12:19,6mPqrKS4AKYJeZ3JLjjubHSwNB96oiJdFX1DgeZNZEBHxM...,TRANSFER,SYSTEM_PROGRAM,success,331670303.0,So11111111111111111111111111111111111111111,1.000000e-09,received,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,...,116.447500,1.164475e-07,Unknown Address,Unknown Address,Unknown Address,Unknown Program,Unknown Entity,Other,Other,Other
1,2025-04-06 00:10:50,29aP1yrCx2dbJM4Mxk8257G5e87bY7CgxySDS6LiAAmsxX...,TRANSFER,SYSTEM_PROGRAM,success,331561835.0,So11111111111111111111111111111111111111111,1.000000e-09,received,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,...,116.447500,1.164475e-07,Unknown Address,Unknown Address,Unknown Address,Unknown Program,Unknown Entity,Other,Other,Other
2,2025-04-06 00:10:45,2D7D4ndxhZfwZ6TRN6pzY7PQEFkRKTQhQjQ1zpz5MyNDVS...,TRANSFER,SYSTEM_PROGRAM,success,331561822.0,So11111111111111111111111111111111111111111,1.000000e-09,received,FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb,...,116.447500,1.164475e-07,Unknown Address,Unknown Address,Unknown Address,Unknown Program,Unknown Entity,Other,Other,Other
3,2025-04-05 23:38:43,nPrzWnG7SLNepcPeBWjEMKadD6WNuWScRgEwEqSNsHiZNN...,TRANSFER,SYSTEM_PROGRAM,success,331557009.0,So11111111111111111111111111111111111111111,1.000000e-05,received,6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV,...,120.140417,1.201404e-03,Unknown Address,Unknown Address,Unknown Address,Unknown Program,Unknown Entity,Other,Other,Other
4,2025-04-05 23:38:25,yqSAMWuNg3pC9H5EZJmTr1MudCuUajZye13BMfaS2atLvC...,TRANSFER,SYSTEM_PROGRAM,success,331556964.0,So11111111111111111111111111111111111111111,1.000000e-09,received,GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg,...,120.140417,1.201404e-07,Unknown Address,Unknown Address,Unknown Address,Unknown Program,Unknown Entity,Other,Other,Other


## Entity Labeling 

In [58]:
import pandas as pd

# Group by sender and receiver to get wallet behaviors
send_stats = test_wallet_timeseries[test_wallet_timeseries['direction'] == 'sent'].groupby('sender').agg({
    'token_amount': 'sum',
    'signature': 'count',
    'receiver_name': pd.Series.nunique
}).rename(columns={
    'token_amount': 'total_sent',
    'signature': 'sent_tx_count',
    'receiver_name': 'unique_receivers'
})

receive_stats = test_wallet_timeseries[test_wallet_timeseries['direction'] == 'received'].groupby('sender').agg({
    'token_amount': 'sum',
    'signature': 'count',
    'receiver_name': pd.Series.nunique
}).rename(columns={
    'token_amount': 'total_received',
    'signature': 'received_tx_count',
    'receiver_name': 'unique_senders'
})

# Combine both
wallet_stats = send_stats.join(receive_stats, how='outer').fillna(0)

# Apply labeling logic
def label_entity(row):
    if row['sent_tx_count'] > 100 and row['received_tx_count'] > 100 and (row['unique_receivers'] + row['unique_senders']) > 50:
        return 'Exchange'
    elif row['sent_tx_count'] > 50 and row['received_tx_count'] < 10 and row['unique_receivers'] > 30:
        return 'Project Wallet'
    elif row['sent_tx_count'] > 10 and row['received_tx_count'] > 10 and row['total_sent'] < 0.01:
        return 'Suspicious'
    else:
        return 'Normal User'

wallet_stats['entity_label'] = wallet_stats.apply(label_entity, axis=1)


In [60]:
# Final structured dataset
labeled_entities_df = wallet_stats.reset_index().rename(columns={'sender': 'wallet_address'})

# Show result
labeled_entities_df[['wallet_address', 'total_sent', 'total_received', 'sent_tx_count', 'received_tx_count', 'unique_receivers', 'entity_label']]

labeled_entities_df.head()

,wallet_address,total_sent,sent_tx_count,unique_receivers,total_received,received_tx_count,unique_senders,entity_label
0,242NMeDWEuM2zTtsJVqJYDbjZsnYDC3iQ2dqZcdoQ6Ai,0.0,0.0,0.0,0.000010,1.0,1.0,Normal User
1,2AEWSvUds1wsufnsDPCXjFsJCMJH5SNNm7fSF4kxys9a,0.0,0.0,0.0,0.437364,1.0,1.0,Normal User
2,2AQdpHJ2JpcEgPiATUXjQxA8QmafFegfQwSLWSprPicm,0.0,0.0,0.0,0.061630,1.0,1.0,Normal User
3,2DNxPzaQkmAxsEzWd4NM1soPr6q6CuB3voPr8f5xNiQc,0.0,0.0,0.0,1.000000,1.0,1.0,Normal User
4,2EEw1A49utRqUsnYRYWtnpz2UQ7n7sJn8EEpdN2nKqWQ,0.0,0.0,0.0,3490.000000,1.0,1.0,Normal User
